In [42]:
include("Create_Sample.jl")
include("MIP_SPLIT.jl")
include("Split.jl")
include("GA.jl")
include("Initial.jl")
include("Mutation.jl")
include("Crossover.jl")
include("Neighborhood.jl")
include("Neighborhood_intra.jl")
include("costs.jl")

Calculate_new_cost_3_permute (generic function with 1 method)

In [50]:
T = Read_TSPLIB_instance(:berlin52, 1)
n_nodes = size(T)[1]-2
demands = ones(Int, n_nodes)
W = 150
h = 0.3
K = 3
tsp_tour = find_tsp_tour1(T[1:n_nodes+1, 1:n_nodes+1])
penalty = 1.1
max_inf = 3
mu = 20
P, f = Generate_initial_population(T, demands, K, W, mu, tsp_tour, penalty, max_inf);

In [51]:
f

3110.882241808066

In [52]:
p1 = deepcopy(P[35])

Chromosome([9, 41, 42, 19, 28, 22, 31, 8, 3, 7  …  12, 23, 51, 50, 18, 11, 1, 30, 46, 34], false, 9639.75281778642, 0.0, Tour[Tour([9, 41, 42, 19, 28, 22, 31, 8, 3, 7, 14, 17, 2, 49], 7372.987920295538), Tour([6, 47, 4, 16, 37, 29, 13, 36, 38, 26, 32, 45, 33], 8239.220412352088), Tour([27, 39, 15, 24, 48, 35, 40, 25, 44, 10, 20, 5, 21, 43], 8344.366526825046), Tour([12, 23, 51, 50, 18, 11, 1, 30, 46, 34], 8489.373689678056)])

In [53]:
Diversify(P, T, demands, K, W, mu, tsp_tour, penalty, max_inf)

In [54]:
p2 = deepcopy(P[35])

Chromosome([19, 3, 10, 25, 30, 22, 29, 50, 33, 16  …  44, 41, 40, 14, 1, 42, 18, 9, 28, 43], false, 6082.139979556951, 0.0, Tour[Tour([19, 3, 10, 25, 30, 22, 29, 50, 33], 4935.36502488842), Tour([16, 8, 21, 37, 6, 45, 23, 17, 4], 4975.378747549967), Tour([11, 5, 36, 20, 2, 7, 47, 34, 13], 5005.821554371104), Tour([26, 31, 15, 32, 27, 49], 4489.497472534836), Tour([48, 39, 24, 51, 46, 12, 35, 38, 44, 41], 4659.478258480136), Tour([40, 14, 1, 42, 18, 9, 28, 43], 5026.561966575992)])

In [32]:
p1.fitness

11657.252965351272

In [33]:
p2.fitness

11657.252965351272

In [35]:
p1.genes == p2.genes

true